# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Nettoyer les données

In [13]:
df_youtube["date"]=pd.to_datetime(df_youtube["date"])
# car vide_id n'est pas un id ici
df_youtube = df_youtube.rename(columns={"video_id": "amphitheatre"})
df_youtube.dtypes

amphitheatre              object
title                     object
channel_title             object
category_id                int64
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_total              int64
thumbnail_link            object
date              datetime64[ns]
dtype: object

In [14]:
# Importation des données
df_json =pd.read_json('./data/US_category_id.json')
#df_json.head(3)

# Création d'un dataframe à partir du dictionnaire afin d'avoir les id
df_json_items = pd.DataFrame(list(df_json['items']))
# On s'assure du type de la colonne id pour le merge, puis renome la colonne 
df_json_items["id"]=pd.to_numeric(df_json_items["id"],downcast='integer')
df_json_items=df_json_items.rename(columns={"id":"category_id"})

# Concatenation des deux dataframes crée çi_dessus
df=pd.concat([df_json, df_json_items], axis=1, sort=False)

# Création d'un nouveau dataframe à partir du second dictionnaire afin d'avoir les snippets
df_json_snippet=pd.DataFrame(list(df_json_items['snippet']))
df_youtube_final= pd.concat([df, df_json_snippet], axis=1, sort=False)

# Supression de la colonne désormais obsolète
df_youtube_final =df_youtube_final.drop(columns=['items','snippet'])

# Jointure sur les colonnes des id des deux dataframes
df_youtube_final = pd.merge(df_youtube,df_youtube_final, on='category_id')
df_youtube_final.tail(10)

,amphitheatre,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,kind,etag,kind,etag,channelId,title_y,assignable
7982,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,294078,8854,237,2019,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,1970-01-01 00:00:00.000000016,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7983,qEp3rwe3KmI,Sphere Treehouses in Canada,INSIDER,19,inspired|treehouse|travel|woods|canada,18419,455,7,56,https://i.ytimg.com/vi/qEp3rwe3KmI/default.jpg,1970-01-01 00:00:00.000000016,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7984,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,342463,9697,261,1849,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,1970-01-01 00:00:00.000000017,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7985,qEp3rwe3KmI,Sphere Treehouses in Canada,INSIDER,19,inspired|treehouse|travel|woods|canada,19606,468,7,58,https://i.ytimg.com/vi/qEp3rwe3KmI/default.jpg,1970-01-01 00:00:00.000000017,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7986,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,366424,10117,264,1927,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,1970-01-01 00:00:00.000000018,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7987,ThPzNokG5uE,Japanese Candy Art,INSIDER,19,Shinobu Ichiyanagi|Japanese|Candy|Art|Amezaiku...,262878,7594,62,663,https://i.ytimg.com/vi/ThPzNokG5uE/default.jpg,1970-01-01 00:00:00.000000019,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7988,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,383835,10457,275,1948,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,1970-01-01 00:00:00.000000019,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7989,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,396811,10633,279,1552,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,1970-01-01 00:00:00.000000020,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/M-3iD9dwK7YJCafRf...",UCBR8-60-B28hp2BmDPdntcQ,Travel & Events,True
7990,5mdk5oz6l3w,Pixel 2: Squeeze to access features (CNET News),CNET,43,Google|Pixel 2|Squeeze|Google Assistant|Phones...,8259,111,8,37,https://i.ytimg.com/vi/5mdk5oz6l3w/default.jpg,1970-01-01 00:00:00.000000006,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/0n9MJVCDLpA8q7aiG...",UCBR8-60-B28hp2BmDPdntcQ,Shows,False
7991,5mdk5oz6l3w

### Importer les données

In [15]:
collection.delete_many({})
collection.insert_many(new_df.to_dict(orient='records'))

## Question 1  

In [16]:
# Récupèration des lignes où le 'channel_title' = 'Apple'.
Apple_list = collection.find({'channel_title' : "Apple"}, {'video_id':1, 'video_title':1, 'channel_title':1})
list(Apple_list)

[{'_id': ObjectId('6020349a54d71741307c5fec'),
  'video_id': 2,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('6020349a54d71741307c5ffc'),
  'video_id': 204,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('6020349a54d71741307c600f'),
  'video_id': 420,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('6020349a54d71741307c6024'),
  'video_id': 675,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('6020349a54d71741307c6036'),
  'video_id': 906,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('6020349a54d71741307c6044'),
  'video_id': 1062,
  'video_title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('6020349a54d71741307c6058'),
  'video_id': 

## Question 2

In [17]:
NbCat = len(collection.distinct('category_id')) # Nombre d'élement dans une liste des différentes catégories (distinctes)
print("Il y a : "+str(NbCat)+" categories différents")

Il y a : 16 categories différents


## Question 3

In [8]:
#cf nettoyage des données

## Question 4

In [18]:
# On trie par nombre de vue décroissant
best_videos = collection.find({}, {'video_id':1, 'video_title':1, 'views':1}).sort([('views', -1)])

# On selectionne les 10 premiers
list(best_videos)[0:10]

[{'_id': ObjectId('6020349a54d71741307c6e6d'),
  'video_id': 1908,
  'video_title': "BTS (방탄소년단) 'DNA' Official MV",
  'views': 41500672},
 {'_id': ObjectId('6020349a54d71741307c6e59'),
  'video_id': 1695,
  'video_title': "BTS (방탄소년단) 'DNA' Official MV",
  'views': 38013692},
 {'_id': ObjectId('6020349a54d71741307c6d44'),
  'video_id': 178,
  'video_title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'views': 36323498},
 {'_id': ObjectId('6020349a54d71741307c5f37'),
  'video_id': 7136,
  'video_title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'views': 33191594},
 {'_id': ObjectId('6020349a54d71741307c5f0b'),
  'video_id': 6918,
  'video_title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'views': 32512343},
 {'_id': ObjectId('6020349a54d71741307c6e73'),
  'video_id': 1996,
  'video_title': 'Shakira - Perro Fiel (Official Video) ft. Nicky Jam',
  'views': 32136948},
 {'_id': ObjectId('6020349a54d71741307c5edd'),
  'video_id': 6673,
  'video_t

## Question 5

In [19]:
# On groupe par catégorie, puis on calcule la moyenne de vue pour chaque catégorie
avg_categories = collection.aggregate([{'$group' : {'_id' : '$category_title', "moyenne de vue": {"$avg" : "$views"}}}])
list(avg_categories)

[{'_id': 'Pets & Animals', 'moyenne de vue': 651404.3879310344},
 {'_id': 'Travel & Events', 'moyenne de vue': 464041.0833333333},
 {'_id': 'Autos & Vehicles', 'moyenne de vue': 607693.8706896552},
 {'_id': 'Science & Technology', 'moyenne de vue': 924730.548828125},
 {'_id': 'Howto & Style', 'moyenne de vue': 537665.1806674339},
 {'_id': 'People & Blogs', 'moyenne de vue': 971532.7573696146},
 {'_id': 'Entertainment', 'moyenne de vue': 1154868.4397251718},
 {'_id': 'Film & Animation', 'moyenne de vue': 1039472.6560846561},
 {'_id': 'Music', 'moyenne de vue': 1176553.612},
 {'_id': 'Education', 'moyenne de vue': 547582.4700598803},
 {'_id': 'News & Politics', 'moyenne de vue': 540955.5569823436},
 {'_id': 'Gaming', 'moyenne de vue': 681081.8170731707},
 {'_id': 'Shows', 'moyenne de vue': 8492.5},
 {'_id': 'Comedy', 'moyenne de vue': 1240073.294039735},
 {'_id': 'Nonprofits & Activism', 'moyenne de vue': 1110334.2142857143},
 {'_id': 'Sports', 'moyenne de vue': 728434.3243902439}]

## Question 6 

In [20]:
# On groupe les videos par le nom de la chaine youtube , puis on calcule la moyenne de like pour chaque chaine
channels_likes = collection.aggregate([
    {'$group' : {'_id' : '$channel_title', 'moyenne de like': { "$avg" : "$likes"}}},
    {'$sort' : {'moyenne de like': -1}}
])
list(channels_likes)[0:10]

[{'_id': 'ZaynVEVO', 'moyenne de like': 1431683.0},
 {'_id': 'ibighit', 'moyenne de like': 1371766.25},
 {'_id': 'melanie martinez', 'moyenne de like': 911871.5},
 {'_id': 'BETNetworks', 'moyenne de like': 769687.625},
 {'_id': 'jypentertainment', 'moyenne de like': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'moyenne de like': 653443.5384615385},
 {'_id': 'Wengie', 'moyenne de like': 567235.2},
 {'_id': 'Alesso', 'moyenne de like': 509417.28571428574},
 {'_id': 'PewDiePie', 'moyenne de like': 508647.8},
 {'_id': 'shakiraVEVO', 'moyenne de like': 434302.14285714284}]